## Imports and Google Drive Auth

In [1]:
!pip install torch numpy transformers datasets tiktoken wandb tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 32.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.8/252.8 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires openai, which is not installed.


In [2]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
%cd /content/drive/MyDrive/Fall/CS229S/cs229s-final-project/

/content/drive/MyDrive/Fall/CS229S/cs229s-final-project


In [4]:
import os
import time
import math
import pickle
from contextlib import nullcontext

import numpy as np
import torch
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.distributed import init_process_group, destroy_process_group

from model import GPTConfig, GPT

import tiktoken

## Debugging + Save Quantization Model

In [5]:
import tiktoken

# -----------------------------------------------------------------------------
init_from = 'resume' # either 'resume' (from an out_dir) or a gpt2 variant (e.g. 'gpt2-xl')
out_dir = 'wikitext' # ignored if init_from is not 'resume'
start = "\n" # or "<|endoftext|>" or etc. Can also specify a file, use as: "FILE:prompt.txt"
num_samples = 10 # number of samples to draw
max_new_tokens = 500 # number of tokens generated in each sample
temperature = 0.8 # 1.0 = no change, < 1.0 = less random, > 1.0 = more random, in predictions
top_k = 200 # retain only the top_k most likely tokens, clamp others to have 0 probability
seed = 1337
device = 'cuda' # examples: 'cpu', 'cuda', 'cuda:0', 'cuda:1', etc.
dtype = 'bfloat16' if torch.cuda.is_available() and torch.cuda.is_bf16_supported() else 'float16' # 'float32' or 'bfloat16' or 'float16'
compile = False # use PyTorch 2.0 to compile the model to be faster
# exec(open('configurator.py').read()) # overrides from command line or config file
# -----------------------------------------------------------------------------

torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cuda.matmul.allow_tf32 = True # allow tf32 on matmul
torch.backends.cudnn.allow_tf32 = True # allow tf32 on cudnn
device_type = 'cuda' if 'cuda' in device else 'cpu' # for later use in torch.autocast
ptdtype = {'float32': torch.float32, 'bfloat16': torch.bfloat16, 'float16': torch.float16}[dtype]
ctx = nullcontext() if device_type == 'cpu' else torch.amp.autocast(device_type=device_type, dtype=ptdtype)

# model
if init_from == 'resume':
    # init from a model saved in a specific directory
    ckpt_path = os.path.join(out_dir, 'ckpt.pt')
    print(f'Loading from {ckpt_path}')
    checkpoint = torch.load(ckpt_path, map_location=device)
    gptconf = GPTConfig(**checkpoint['model_args'])
    model = GPT(gptconf)
    state_dict = checkpoint['model']
    unwanted_prefix = '_orig_mod.'
    for k,v in list(state_dict.items()):
        if k.startswith(unwanted_prefix):
            state_dict[k[len(unwanted_prefix):]] = state_dict.pop(k)
    model.load_state_dict(state_dict)
elif init_from.startswith('gpt2'):
    # init from a given GPT-2 model
    model = GPT.from_pretrained(init_from, dict(dropout=0.0))

model.eval()
model.to(device)
if compile:
    model = torch.compile(model) # requires PyTorch 2.0 (optional)

# look for the meta pickle in case it is available in the dataset folder
load_meta = False
if init_from == 'resume' and 'config' in checkpoint and 'dataset' in checkpoint['config']: # older checkpoints might not have these...
    meta_path = os.path.join('data', checkpoint['config']['dataset'], 'meta.pkl')
    load_meta = os.path.exists(meta_path)
if load_meta:
    print(f"Loading meta from {meta_path}...")
    with open(meta_path, 'rb') as f:
        meta = pickle.load(f)
    # TODO want to make this more general to arbitrary encoder/decoder schemes
    stoi, itos = meta['stoi'], meta['itos']
    encode = lambda s: [stoi[c] for c in s]
    decode = lambda l: ''.join([itos[i] for i in l])
else:
    # ok let's assume gpt-2 encodings by default
    print("No meta.pkl found, assuming GPT-2 encodings...")
    enc = tiktoken.get_encoding("gpt2")
    encode = lambda s: enc.encode(s, allowed_special={"<|endoftext|>"})
    decode = lambda l: enc.decode(l)

# encode the beginning of the prompt
if start.startswith('FILE:'):
    with open(start[5:], 'r', encoding='utf-8') as f:
        start = f.read()
start_ids = encode(start)
x = (torch.tensor(start_ids, dtype=torch.long, device=device)[None, ...])


Loading from wikitext/ckpt.pt
Model in quantized mode
number of parameters: 123.65M
No meta.pkl found, assuming GPT-2 encodings...


In [6]:
total_bytes = 0
for param in model.parameters():
    total_bytes += param.numel() * param.element_size()

print(f"Total memory usage in bytes: {total_bytes}")
print(f"Total memory usage in megabytes: {total_bytes / (1024 ** 2)}")
print(param.element_size())

for name, param in model.named_parameters():
    print(f"Layer: {name}, Type: {param.dtype}, Size: {param.data.shape}")

Total memory usage in bytes: 497759232
Total memory usage in megabytes: 474.7001953125
4
Layer: transformer.wte.weight, Type: torch.float32, Size: torch.Size([50257, 768])
Layer: transformer.wpe.weight, Type: torch.float32, Size: torch.Size([1024, 768])
Layer: transformer.h.0.ln_1.weight, Type: torch.float32, Size: torch.Size([768])
Layer: transformer.h.0.ln_1.bias, Type: torch.float32, Size: torch.Size([768])
Layer: transformer.h.0.attn.c_attn.weight, Type: torch.float32, Size: torch.Size([2304, 768])
Layer: transformer.h.0.attn.c_attn.bias, Type: torch.float32, Size: torch.Size([2304])
Layer: transformer.h.0.attn.c_proj.weight, Type: torch.float32, Size: torch.Size([768, 768])
Layer: transformer.h.0.attn.c_proj.bias, Type: torch.float32, Size: torch.Size([768])
Layer: transformer.h.0.ln_2.weight, Type: torch.float32, Size: torch.Size([768])
Layer: transformer.h.0.ln_2.bias, Type: torch.float32, Size: torch.Size([768])
Layer: transformer.h.0.mlp.c_fc.weight, Type: torch.float32, Size:

In [7]:
model.quantize_transformers()
print(model.quantization_scales)

{'transformer.h.0.ln_1.weight': tensor(501.8847, device='cuda:0'), 'transformer.h.0.ln_1.bias': tensor(490.1018, device='cuda:0'), 'transformer.h.0.attn.c_attn.weight': tensor(44.7227, device='cuda:0'), 'transformer.h.0.attn.c_attn.bias': tensor(94.9118, device='cuda:0'), 'transformer.h.0.attn.c_proj.weight': tensor(38.4227, device='cuda:0'), 'transformer.h.0.attn.c_proj.bias': tensor(47.3122, device='cuda:0'), 'transformer.h.0.ln_2.weight': tensor(83.9078, device='cuda:0'), 'transformer.h.0.ln_2.bias': tensor(171.3022, device='cuda:0'), 'transformer.h.0.mlp.c_fc.weight': tensor(27.8004, device='cuda:0'), 'transformer.h.0.mlp.c_fc.bias': tensor(170.8822, device='cuda:0'), 'transformer.h.0.mlp.c_proj.weight': tensor(20.7393, device='cuda:0'), 'transformer.h.0.mlp.c_proj.bias': tensor(85.9377, device='cuda:0'), 'transformer.h.1.ln_1.weight': tensor(192.7835, device='cuda:0'), 'transformer.h.1.ln_1.bias': tensor(190.9859, device='cuda:0'), 'transformer.h.1.attn.c_attn.weight': tensor(103.

In [24]:
# post quantization

total_bytes = 0
for param in model.parameters():
    total_bytes += param.numel() * param.element_size()

print(f"Total memory usage in bytes: {total_bytes}")
print(f"Total memory usage in megabytes: {total_bytes / (1024 ** 2)}")

for name, param in model.named_parameters():
    print(f"Layer: {name}, Type: {param.dtype}, Size: {param.data.shape}")

Total memory usage in bytes: 242595840
Total memory usage in megabytes: 231.357421875
4
Layer: transformer.wte.weight, Type: torch.float32, Size: torch.Size([50257, 768])
Layer: transformer.wpe.weight, Type: torch.float32, Size: torch.Size([1024, 768])
Layer: transformer.h.0.ln_1.weight, Type: torch.int8, Size: torch.Size([768])
Layer: transformer.h.0.ln_1.bias, Type: torch.int8, Size: torch.Size([768])
Layer: transformer.h.0.attn.c_attn.weight, Type: torch.int8, Size: torch.Size([2304, 768])
Layer: transformer.h.0.attn.c_attn.bias, Type: torch.int8, Size: torch.Size([2304])
Layer: transformer.h.0.attn.c_proj.weight, Type: torch.int8, Size: torch.Size([768, 768])
Layer: transformer.h.0.attn.c_proj.bias, Type: torch.int8, Size: torch.Size([768])
Layer: transformer.h.0.ln_2.weight, Type: torch.int8, Size: torch.Size([768])
Layer: transformer.h.0.ln_2.bias, Type: torch.int8, Size: torch.Size([768])
Layer: transformer.h.0.mlp.c_fc.weight, Type: torch.int8, Size: torch.Size([3072, 768])
Lay

In [9]:
model.quantization_scales

{'transformer.h.0.ln_1.weight': tensor(501.8847, device='cuda:0'),
 'transformer.h.0.ln_1.bias': tensor(490.1018, device='cuda:0'),
 'transformer.h.0.attn.c_attn.weight': tensor(44.7227, device='cuda:0'),
 'transformer.h.0.attn.c_attn.bias': tensor(94.9118, device='cuda:0'),
 'transformer.h.0.attn.c_proj.weight': tensor(38.4227, device='cuda:0'),
 'transformer.h.0.attn.c_proj.bias': tensor(47.3122, device='cuda:0'),
 'transformer.h.0.ln_2.weight': tensor(83.9078, device='cuda:0'),
 'transformer.h.0.ln_2.bias': tensor(171.3022, device='cuda:0'),
 'transformer.h.0.mlp.c_fc.weight': tensor(27.8004, device='cuda:0'),
 'transformer.h.0.mlp.c_fc.bias': tensor(170.8822, device='cuda:0'),
 'transformer.h.0.mlp.c_proj.weight': tensor(20.7393, device='cuda:0'),
 'transformer.h.0.mlp.c_proj.bias': tensor(85.9377, device='cuda:0'),
 'transformer.h.1.ln_1.weight': tensor(192.7835, device='cuda:0'),
 'transformer.h.1.ln_1.bias': tensor(190.9859, device='cuda:0'),
 'transformer.h.1.attn.c_attn.weight

In [10]:
# trial run of post quantization inference

model.quantize = True
torch.cuda.reset_max_memory_allocated()

start_time = time.time()
y = model.generate(x, max_new_tokens, temperature=temperature, top_k=top_k)
print(decode(y[0].tolist()))
print('---------------')

print(f"Took {time.time() - start_time} seconds")
print(f"Memory used for inference with quantization: {torch.cuda.max_memory_allocated()} bytes")

/usr/local/lib/python3.10/dist-packages/torch/cuda/memory.py:329: FutureWarning: torch.cuda.reset_max_memory_allocated now calls torch.cuda.reset_peak_memory_stats, which resets /all/ peak memory stats.
  warnings.warn(



- * * AllMusic 
<|endoftext|><|endoftext|><|endoftext|><|endoftext|> = = Historiography and studies = = 
<|endoftext|> = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = 

In [11]:
state_dict = model.state_dict()
total_bytes = 0

for key, tensor in state_dict.items():
    total_bytes += tensor.numel() * tensor.element_size()

print(f"Total memory usage of state_dict in bytes: {total_bytes}")
print(f"Total memory usage of state_dict in kilobytes: {total_bytes / 1024}")
print(f"Total memory usage of state_dict in megabytes: {total_bytes / (1024 ** 2)}")

Total memory usage of state_dict in bytes: 396985344
Total memory usage of state_dict in kilobytes: 387681.0
Total memory usage of state_dict in megabytes: 378.5947265625


In [12]:
quantize_checkpoint = {
    'model': model.state_dict(),
    'model_args': checkpoint['model_args'],
    'quantization_scales' : model.quantization_scales
}
torch.save(quantize_checkpoint, os.path.join(out_dir, 'quantized_ckpt.pt'))

## Quantized Model Testing

In [26]:
import tiktoken

# -----------------------------------------------------------------------------
init_from = 'resume' # either 'resume' (from an out_dir) or a gpt2 variant (e.g. 'gpt2-xl')
out_dir = 'wikitext' # ignored if init_from is not 'resume'
start = "Say something cool:" # or "<|endoftext|>" or etc. Can also specify a file, use as: "FILE:prompt.txt"
num_samples = 10 # number of samples to draw
max_new_tokens = 500 # number of tokens generated in each sample
temperature = 0.8 # 1.0 = no change, < 1.0 = less random, > 1.0 = more random, in predictions
top_k = 200 # retain only the top_k most likely tokens, clamp others to have 0 probability
seed = 1337
device = 'cuda' # examples: 'cpu', 'cuda', 'cuda:0', 'cuda:1', etc.
dtype = 'bfloat16' if torch.cuda.is_available() and torch.cuda.is_bf16_supported() else 'float16' # 'float32' or 'bfloat16' or 'float16'
compile = False # use PyTorch 2.0 to compile the model to be faster
# exec(open('configurator.py').read()) # overrides from command line or config file
# -----------------------------------------------------------------------------

torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cuda.matmul.allow_tf32 = True # allow tf32 on matmul
torch.backends.cudnn.allow_tf32 = True # allow tf32 on cudnn
device_type = 'cuda' if 'cuda' in device else 'cpu' # for later use in torch.autocast
ptdtype = {'float32': torch.float32, 'bfloat16': torch.bfloat16, 'float16': torch.float16}[dtype]
ctx = nullcontext() if device_type == 'cpu' else torch.amp.autocast(device_type=device_type, dtype=ptdtype)

# initialize quantized.model
# init from a model saved in a specific directory
ckpt_path = os.path.join(out_dir, 'quantized_ckpt.pt')
checkpoint = torch.load(ckpt_path, map_location=device)
gptconf = GPTConfig(**checkpoint['model_args'])
model = GPT(gptconf)
state_dict = checkpoint['model']
unwanted_prefix = '_orig_mod.'
# format params
for k,v in list(state_dict.items()):
    if k.startswith(unwanted_prefix):
        state_dict[k[len(unwanted_prefix):]] = state_dict.pop(k)
# add params
for name, param in model.named_parameters():
    if 'transformer.h' in name:
      param.requires_grad = False
      param.data = param.data.to(torch.int8)
model.load_state_dict(state_dict)
# add quantization scales
model.quantization_scales = checkpoint['quantization_scales']
print(model.quantization_scales)



# switch to eval
model.eval()
model.to(device)
if compile:
    model = torch.compile(model) # requires PyTorch 2.0 (optional)

# ok let's assume gpt-2 encodings by default
print("No meta.pkl found, assuming GPT-2 encodings...")
enc = tiktoken.get_encoding("gpt2")
encode = lambda s: enc.encode(s, allowed_special={"<|endoftext|>"})
decode = lambda l: enc.decode(l)

# encode the beginning of the prompt
if start.startswith('FILE:'):
    with open(start[5:], 'r', encoding='utf-8') as f:
        start = f.read()
start_ids = encode(start)
x = (torch.tensor(start_ids, dtype=torch.long, device=device)[None, ...])


Model in quantized mode
number of parameters: 123.65M
{'transformer.h.0.ln_1.weight': tensor(501.8847, device='cuda:0'), 'transformer.h.0.ln_1.bias': tensor(490.1018, device='cuda:0'), 'transformer.h.0.attn.c_attn.weight': tensor(44.7227, device='cuda:0'), 'transformer.h.0.attn.c_attn.bias': tensor(94.9118, device='cuda:0'), 'transformer.h.0.attn.c_proj.weight': tensor(38.4227, device='cuda:0'), 'transformer.h.0.attn.c_proj.bias': tensor(47.3122, device='cuda:0'), 'transformer.h.0.ln_2.weight': tensor(83.9078, device='cuda:0'), 'transformer.h.0.ln_2.bias': tensor(171.3022, device='cuda:0'), 'transformer.h.0.mlp.c_fc.weight': tensor(27.8004, device='cuda:0'), 'transformer.h.0.mlp.c_fc.bias': tensor(170.8822, device='cuda:0'), 'transformer.h.0.mlp.c_proj.weight': tensor(20.7393, device='cuda:0'), 'transformer.h.0.mlp.c_proj.bias': tensor(85.9377, device='cuda:0'), 'transformer.h.1.ln_1.weight': tensor(192.7835, device='cuda:0'), 'transformer.h.1.ln_1.bias': tensor(190.9859, device='cuda:

In [27]:
model.quantize = True
torch.cuda.reset_max_memory_allocated()

start_time = time.time()
y = model.generate(x, max_new_tokens, temperature=temperature, top_k=top_k)
print(decode(y[0].tolist()))
print('---------------')

print(f"Took {time.time() - start_time} seconds")
print(f"Memory used for inference with quantization: {torch.cuda.max_memory_allocated()} bytes")

Say something cool: " * " All four 
<|endoftext|><|endoftext|><|endoftext|><|endoftext|> = = = = = = 2013 : A. E. W. Johnson = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = =

## Perplexity and Memory Usage

## Speculative Decoding